In [1]:
import os, sys, gc, math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from glob import glob
from progressbar import progressbar

matplotlib.rcParams['figure.dpi'] = 600

In [2]:
ANIM_OUT_DIR = "animation_output"
FRAME_PREFIX = "frame"
MAX_ARCHIVED_ANIMS = 10000

In [3]:
def mandel_iter(c, z=0):
    return z**2 + c

In [4]:
def burning_ship_iter(c, z):
    return (abs(z.real) + 1j*abs(z.imag))**2 + c

In [5]:
def iter_diverge(iter_func, c, iterations, bound_radius):
    z = 0
    
    for i in range(iterations):
        z = iter_func(c, z)
        if abs(z) > bound_radius:
            return (True, i, z)
        
    return (False, iterations, z)

In [6]:
def iter_array(iter_func, left_x, right_x, bottom_y, top_y, num_pts, iterations, bound_radius):
    results = np.zeros((num_pts, num_pts))
    
    for x_i, x in enumerate(progressbar(np.linspace(left_x, right_x, num_pts))):
        for y_i, y in enumerate(np.linspace(top_y, bottom_y, num_pts)):
            results[x_i, y_i] = iter_diverge(iter_func, complex(x, y), iterations, bound_radius)[1]
            
    return results

In [7]:
def make_iter_frac(iter_func=mandel_iter, center=complex(0, 0), size=2.0, resolution=100, iterations=50, bound_radius=3.0):
    half = size / 2.0
    left = center.real-half
    right = center.real+half
    bottom = center.imag-half
    top = center.imag+half
    
    num_pts = resolution
    arr = iter_array(iter_func, left, right, bottom, top, num_pts, iterations, bound_radius)
    return arr, (left, right, bottom, top)

In [24]:
def _checkflip(arr, extent, flip_x, flip_y):
    l, r, b, t = extent
    if flip_x:
        arr = np.flip(arr, 0)
        extent = (r, l, b, t)
        l, r = r, l
    if flip_y:
        arr = np.flip(arr, 1)
        extent = (l, r, t, b)
        b, t = t, b
    return arr, extent

In [25]:
def mandel_view(center=complex(0, 0), 
                size=2.0, 
                resolution=100, 
                iterations=50, 
                bound_radius=2.0, 
                color_map='viridis',
                      flip_x = False,
                      flip_y = False):
    return custom_view(mandel_iter, center, size, resolution, iterations, bound_radius, color_map, flip_x, flip_y)

In [26]:
def burning_ship_view(center=complex(0, 0), 
                      size=2.0, 
                      resolution=100, 
                      iterations=1000, 
                      bound_radius=4.0, 
                      color_map='viridis',
                      flip_x = False,
                      flip_y = False):
    return custom_view(burning_ship_iter, center, size, resolution, iterations, bound_radius, color_map, flip_x, flip_y)

In [27]:
def custom_view(iter_func, 
                center=complex(0, 0),
                size=2.0, 
                resolution=100, 
                iterations=50, 
                bound_radius=2.0, 
                color_map='viridis',
                flip_x = False,
                flip_y = False):
    arr, extent = make_iter_frac(iter_func, center, size, resolution, iterations, bound_radius)
    arr, extent = _checkflip(arr, extent, flip_x, flip_y)
    plt.imshow(arr.T, extent=extent, cmap=color_map)
    return arr

In [28]:
def animate_iter_frac(iter_func=mandel_iter, 
                      center_start=complex(0, 0), 
                      size_exp_start=1, 
                      center_end=complex(-0.5, 0), 
                      size_exp_end=-15, 
                      num_frames=100,
                      resolution=200, 
                      iterations=50, 
                      bound_radius=2.0, 
                      color_map='viridis',
                      start_frame=0,
                      movement_type='geom',
                      zoom_type='geom',
                      flip_x = False,
                      flip_y = False):
    
    if zoom_type == 'geom': # geometrically slowing zoom
        sizes = np.logspace(size_exp_start, size_exp_end, num_frames)
    else: # linear zoom 
        sizes = np.linspace(10**size_exp_start, 10**size_exp_end, num_frames)
        
    if movement_type == 'geom': # geometrically slowing pan
        seq = np.logspace(0, size_exp_end, num_frames)
    else: # linear pan
        seq = np.linspace(0, 1, num_frames)
    line = center_end - center_start
    centers = center_end - line*seq

    for count, (cen, siz) in enumerate(zip(centers, sizes)):
        if count >= start_frame-1:
            print("processing frame {} of {}".format(count+1, num_frames))
            sys.stdout.flush()
            arr, extent = make_iter_frac(iter_func, cen, siz, resolution, iterations, bound_radius)
            f = plt.figure()
            plt.imshow(arr, extent=extent, cmap=color_map)
            plt.savefig(os.path.join(ANIM_OUT_DIR, "{}_{}".format(FRAME_PREFIX, count)))
            plt.close(f)
            
            plt.close(1); plt.close('all'); gc.collect() # fix leak maybe??

In [29]:
def archive_frames(): 
    dirs = [d for d in os.listdir(ANIM_OUT_DIR) if os.path.isdir(os.path.join(ANIM_OUT_DIR, d))]
    index = 0
    new_dir_name = None
    while index < MAX_ARCHIVED_ANIMS: # find the first integer that isn't already taken as a frame dir
        index += 1
        if str(index) not in dirs:
            new_dir_name = os.path.join(ANIM_OUT_DIR, str(index))
            break
    if new_dir_name is None:
        raise Exception()
        
    os.mkdir(new_dir_name)
    frame_files = glob(os.path.join(ANIM_OUT_DIR, "{}_*.png".format(FRAME_PREFIX)))
    [os.rename(f, os.path.join(new_dir_name, os.path.basename(f))) for f in frame_files] #move the files

In [30]:
def new_iterfunc(z, c):
    return mandel_iter(z, c) * burning_ship_iter(z, c)

In [ ]:
burning_ship_view(center=complex(-1.5363, -0.000125), 
                  size=0.0002, 
                  resolution=600, 
                  flip_y = True, 
                  bound_radius=30)

 91% (546 of 600) |####################  | Elapsed Time: 0:01:48 ETA:   0:00:11

In [ ]:
#archive_frames()

In [ ]:
animate_iter_frac(iter_func =      burning_ship_iter, 
                  center_start =   complex(0, 0),
                  size_exp_start = 1,
                  center_end =     complex(-0.811343, -0.375323),
                  size_exp_end =   -10,
                  num_frames =     300,
                  resolution =     800,
                  iterations =     300,
                  bound_radius =   50,
                  color_map =      'gist_stern',
                  start_frame =    198,
                  movement_type =  'geom')